# xgboostを用いて、スプライスされやすい領域とそうでない領域の配列の学習を行う

## 準備

In [ ]:
# オリジナルモジュールのインポート
from lib.introngap import PileUp
from lib.gbkparse import Seq_count

# モジュールのインポート
import itertools
import logomaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# クラスのインスタンス化
gbk = Seq_count()

# gbkファイルの読み込み
gbk.read_gbk('../data/gbk/human_ttn.gb')

# 各イントロンの両端n塩基を抽出し下記の条件でデータを分割する
スプライシングを受けやすい領域(highs): intron 120-220<br>
スプライシングを受けにくい領域(lows): intron 221-

## 各イントロンの5-prime側のn塩基にフォーカスし解析

In [ ]:
# 各イントロンの両端n塩基を抽出し
n = 30

# 各イントロンの5-prime側のn塩基を抽出 
left_edges = []
for i in range(gbk.intron_num()):
    left_edges.append(str(gbk.intron_seq(i+1)[:n]))

In [ ]:
# 領域の切り出し
high_s, high_e = 120, 220
low_s, low_e = 0, 45
# low_s, low_e = 220, gbk.intron_num()
left_edges_high = left_edges[high_s:high_e]
left_edge_low = left_edges[low_s:low_e] 

# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences_high = np.array([one_hot_encode(seq) for seq in left_edges_high])
encoded_sequences_low = np.array([one_hot_encode(seq) for seq in left_edge_low])

In [ ]:
# 学習データとしてワンホとエンコーディングされた配列を結合しXとする
# ラベルをyとして結合する
X = pd.DataFrame(np.concatenate([encoded_sequences_high, encoded_sequences_low]))
y = pd.DataFrame(np.concatenate([np.ones(len(encoded_sequences_high)), np.zeros(len(encoded_sequences_low))]))

# XGBoostのインスタンス化
model = XGBClassifier(early_stopping_rounds=10, learning_rate=0.05, max_depth=8)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# データをモデルにfitさせる
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)    

In [ ]:
# logomakerを用いてモチーフを可視化
b_df_r = pd.DataFrame(model.feature_importances_.reshape(30,4))
b_df_r.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(b_df_r, shade_below=.5, fade_below=.5)

In [ ]:
g_count = 0
for i in left_edges_high:
    if i[3] == "A":
        g_count += 1
print(g_count/len(left_edges_high))

g_count = 0
for i in left_edge_low:
    if i[3] == "A":
        g_count += 1
print(g_count/len(left_edge_low))

## 各イントロンの5-prime側のn塩基にフォーカスし解析

In [ ]:
    
# 各イントロンの3-prime側のn塩基を抽出
right_edges = []
for i in range(gbk.intron_num()):
    right_edges.append(str(gbk.intron_seq(i+1)[-n:]))

In [ ]:
# 領域の切り出し
high_s, high_e = 120, 220
low_s, low_e = 0, 45
# low_s, low_e = 220, gbk.intron_num()
right_edges_high = right_edges[high_s:high_e]
right_edges_low = right_edges[low_s:low_e] 

# ワンホットエンコーディングを行う関数
def one_hot_encode(seq):
    mapping = {'A': [1, 0, 0, 0], 'T': [0, 1, 0, 0], 'G': [0, 0, 1, 0], 'C': [0, 0, 0, 1]}
    return np.array([mapping[s] for s in seq]).flatten()

# エンコーディングされた配列を準備
encoded_sequences_high = np.array([one_hot_encode(seq) for seq in right_edges_high])
encoded_sequences_low = np.array([one_hot_encode(seq) for seq in right_edge_low])

In [ ]:
# 学習データとしてワンホとエンコーディングされた配列を結合しXとする
# ラベルをyとして結合する
X = pd.DataFrame(np.concatenate([encoded_sequences_high, encoded_sequences_low]))
y = pd.DataFrame(np.concatenate([np.ones(len(encoded_sequences_high)), np.zeros(len(encoded_sequences_low))]))

# XGBoostのインスタンス化
model = XGBClassifier(early_stopping_rounds=10, learning_rate=0.1, max_depth=8)

# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# データをモデルにfitさせる
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)    

In [ ]:
# logomakerを用いてモチーフを可視化
b_df_r = pd.DataFrame(model.feature_importances_.reshape(30,4))
b_df_r.columns = ['A','T','G','C']
crp_logo = logomaker.Logo(b_df_r, shade_below=.5, fade_below=.5)

In [ ]:
g_count = 0
for i in right_edges_high:
    print(i)
    if i[27] == "A":
        g_count += 1
print(g_count/len(right_edges_high))

In [ ]:

g_count = 0
for i in right_edges_low:
    print(i)
    if i[27] == "A":
        g_count += 1
print(g_count/len(right_edge_low))